In [1]:
import numpy as np
import pandas as pd 

In [2]:
df=pd.read_csv(r"C:\Users\DELL\Desktop\athlete_events.csv")
region_df=pd.read_csv(r"C:\Users\DELL\Desktop\noc_regions.csv")

In [3]:
df.head(5)

In [4]:
region_df.head()

# Filtering the data for summer season 

In [5]:
df=df[df['Season']=='Summer']

In [6]:
df.head()
# df3.drop(columns=['region_x','region_y','notes_y','notes_x'],inplace=True)

# Merging the data df and region_df

In [7]:
df=df.merge(region_df,on="NOC",how='left')

In [8]:
df.head()


# Number of country paricipated 

In [9]:
df['region'].unique()

# Counting Missing and duplicates values

In [10]:
df.isnull().sum()

In [11]:
df.duplicated().sum()

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
df.duplicated().sum()

# Medal Count on the basis of region

In [14]:
df['Medal'].value_counts()

In [15]:
# creating a dummy dataframe 
dummy=pd.get_dummies(df['Medal'])
dummy

In [16]:
#concatanating dummy dataframe and orginal dataframe
df=pd.concat([df,dummy],axis=1)
df.head()

In [17]:
df.groupby('NOC').sum()[["Gold","Silver","Bronze"]]

In [18]:
df.head()

In [19]:
df[(df['NOC']=="IND") & (df['Medal']=='Gold')].head()

In [20]:
medal_tally=df.drop_duplicates(subset=['Team','NOC','Games','Year','City','Sport','Event','Medal'])

In [21]:
medal_tally=medal_tally.groupby('NOC').sum()[["Gold","Silver","Bronze"]].sort_values('Gold',ascending=False).reset_index()

In [22]:
medal_tally[medal_tally['NOC']=='IND']

In [23]:
medal_tally['Total']=medal_tally['Gold']+medal_tally['Silver']+medal_tally['Bronze']

In [24]:
medal_tally[medal_tally['NOC']=='IND']

# Years when olympics were played

In [25]:
year=df['Year'].unique().tolist()
year.sort()
year

# Country where olympics were played

In [26]:
country=df['region'].unique().tolist()

In [27]:
len(country)

In [28]:
c1=np.unique(df['region'].dropna().values).tolist()

In [29]:
c1.sort()

In [30]:
# c1

In [31]:
medal_df=df.drop_duplicates(subset=['Team','NOC','Games','Year','City','Sport','Event','Medal'])

In [32]:
def fetch_model(year,country):
    flag=0
    if year == 'overall' and country == 'overall':
        temp_df=medal_df
    if year == 'overall' and country != 'overall':
        temp_df=medal_df[medal_df['region']==country]
        flag=1
    if year != 'overall' and country == 'overall':
        temp_df=medal_df[medal_df['Year']==int(year)]
    if year != 'overall' and country != 'overall':
        temp_df=medal_df[(medal_df['Year']==int(year)) & (medal_df['region']==country)]
    if flag==1:    
        X=temp_df.groupby('Year').sum()[['Gold','Silver','Bronze']].sort_values('Year').reset_index()
    else:
        X=temp_df.groupby('region').sum()[['Gold','Silver','Bronze']].sort_values('Gold').reset_index()
        
    X['total']=X['Gold']+X['Silver']+X['Bronze']
    print(X)
         

In [33]:
year=input("Enter Year")
country=input("Enter Country")
fetch_model(year,country)

Enter Year2016
Enter CountryIndia


# No of Sports / Event / City / Years / Athletes Name / Region 

In [34]:
df['Sport'].unique()

In [35]:
df['Sport'].unique().shape[0]

In [36]:
df['Event'].unique().shape

In [37]:
df['City'].unique().shape[0]

In [38]:
df["Year"].unique().shape[0]-1        # 1906 olympic was invalid 

In [39]:
df['Name'].unique().shape

In [40]:
df['region'].unique().shape

In [41]:
No. of participating country over time 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
nation_over_time=df.drop_duplicates(["Year","region"])['Year'].value_counts().reset_index().sort_values('Year')

In [ ]:
nation_over_time.rename(columns={'count':'No. of Country'},inplace=True)

In [ ]:
plt.plot(nation_over_time['Year'],nation_over_time['No. of Country'])
plt.xlabel("Years")
plt.ylabel("No. of Country")
plt.show()

# No. of events over time 

In [ ]:
events_count=df.drop_duplicates(["Year","Event"])['Year'].value_counts().reset_index().sort_values('Year')
events_count.rename(columns={'count':'No. of Event'},inplace=True)

In [ ]:
plt.plot(events_count['Year'],events_count['No. of Event'],color='g')
plt.xlabel("Years")
plt.ylabel("No. of Events")
plt.show()

# No. of athletes over time

In [ ]:
athletes_over_time=df.drop_duplicates(["Year","Name"])['Year'].value_counts().reset_index().sort_values('Year')
athletes_over_time.rename(columns={"count":"Total Athletes"},inplace=True)

In [ ]:
plt.plot(athletes_over_time['Year'],athletes_over_time['Total Athletes'],color='g')
plt.xlabel("Years")
plt.ylabel("Total Athletes")
plt.show()

# No. of Sports  over year

In [ ]:
X=df.drop_duplicates(["Year","Sport","Event"])

In [ ]:
import seaborn as sns
plt.figure(figsize=(20,20))
sns.heatmap(X.pivot_table(index='Sport',columns="Year",values='Event',aggfunc="count").fillna(0).astype(int),annot=True)

# List of Successful athletes 

In [ ]:
def successful(df,sport):
    temp_df2=df.dropna(subset=['Medal'])
    if sport!="overall":
        temp_df2 = temp_df2[temp_df2["Sport"]==sport]     
    X=temp_df2['Name'].value_counts().reset_index()
    return X.rename(columns={'count':'Total Medals'}).head(20)

In [ ]:
successful(df,'overall')

# Country wise Medal tally per year

In [ ]:
temp_df=df.dropna(subset=["Medal"])
temp_df.drop_duplicates(subset=['Team','NOC','Games','Year','City','Sport','Event','Medal'],inplace=True)

In [ ]:
new_df=temp_df[temp_df['region']=='India']
final_df=new_df.groupby("Year").count()['Medal'].reset_index()

In [ ]:
final_df

In [ ]:
plt.plot(final_df['Year'],final_df['Medal'],color='g')
plt.xlabel("Years")
plt.ylabel("Total Medals")
plt.show()

# which country is good at heatmap

In [ ]:
new_df=temp_df[temp_df['region']=='UK']
plt.figure(figsize=(15,15))
sns.heatmap(new_df.pivot_table(index='Sport',columns='Year',values='Medal',aggfunc='count').fillna(0),annot=True)

# Analysis of Height vs Weight in different- different sports 

In [ ]:
athlete_df=df.drop_duplicates(subset=['Name','region'])

In [ ]:
athlete_df['Medal'].fillna("No Medal",inplace=True)

In [ ]:
temp_df=athlete_df[athlete_df['Sport']=="Athletics"]

In [ ]:
plt.figure(figsize=(15,10))
sns.scatterplot(x=temp_df['Weight'],y=temp_df['Height'],hue=temp_df['Medal'],style=temp_df['Sex'],s=80)

# Participation of Men Vs Women 

In [ ]:
Men=athlete_df[athlete_df['Sex']=="M"].groupby("Year").count()['Name'].reset_index()
Women=athlete_df[athlete_df['Sex']=="F"].groupby("Year").count()['Name'].reset_index()

In [ ]:
final=Men.merge(Women,on='Year',how='left',inplace=True)

In [ ]:
final.rename(columns={'Name_x':'Men','Name_y':'Women'},inplace=True)
final.head()

In [ ]:
final.fillna(0,inplace=True)

In [ ]:
final.astype(int).head()

In [ ]:
plt.plot(final['Year'],final['Men'],label="Men")
plt.plot(final['Year'],final['Women'],label='Women')
plt.xlabel("Year")
plt.ylabel("Total Players")
plt.legend()